## 27. GANのおける工夫

### 評価指標
最もよく使われ受け入れられている評価指標に
- インセプション・スコア(Inception Score: IS)
- フレチェのインセプション距離(Frechent Inception Distance: FID)

がある。

#### インセプション・スコア
評価指標が保証すべき要求には、
- 生成サンプルは識別できるもののように見えること。  
それはリアルであり、データセット内に存在するなにかの物体を含んで生成されること。  
さらに画像分類器は、入力される画像内に認識できるものがあれば確実に見つけられること。  
画像が特定のクラスに属することを確信度と共に出力できる分類器にインセプション・ネットワークがあり、ここからスコアの名前がとられている。  
  
- 生成されるサンプルは、元々のデータセットに含まれるクラスを図べ手ふくんでいるのが理想。  
クラス間のモード崩壊は望まれない。  
  
などがある。  
  
ISの計算は、以下のように単純である。  
1. 本物と偽物の分布との間でKLダイバージェンスを計算する  
2. このべき乗をとる
  
#### フレチェのインセプション距離
もしISで実現される最低限の性能で満足してよいなら、1カテゴリだけを生成するようなものでも、  
それがカテゴリにマッチするものでありさえすれば高性能とされる。  
FIDは画像をインセプション・ネットワークを通しながら計算される。  
最終的なが画像ではなく、中間表現（特徴マップや層）を比較する。

#### 訓練における課題
- モード崩壊  
　いくつかのモードが生成されるサンプルに含まれなくなる。  
- 収束の遅さ  
- 過度な汎化  
　モード(生成されうるサンプル)の中に、サポートが存在しないものが含まれてしまう状況。  
  
以下のような様々な技術によって改善することができる。  
- ネットワークを深くしていく
- ゲームの設定を変える  
　・ Min-Max方式と停止基準が元論文で提案されている  
　・ 非飽和(Non-Saturating)方式と停止基準も元論文で提案されている  
　・ Wassertein GANという改善策もある
- 論争はあるが、訓練に使えるいくつかのハックがある  
　・ 入力の正規化  
　・ 勾配の制約  
　・ 識別器をより多く訓練する  
　・ 疎な勾配を避ける  
　・ ソフトな、あるいはノイズ付きラベルに切り替える  

#### Min-Max GAN
GANを**min-maxゲーム**(最大の損失が最小になるように決断を行う戦略)と捉える方法。  
識別器のスコア関数は、
$$
J^D=E_{x\sim p_r}\log{[D(x)]}+E_{x\sim p_g}\log{[1-D(G(x))]}
$$

$E$は$x$(本物のデータ分布)または$z$(潜在空間)全体にわたる期待値を表す。  
$D$は識別器を表す関数、$G$は生成器を表す関数である。  
上の式は、二値分類問題においてよく使われる形態であり、この式を自由に単純化すると次のようになる。  
$$
J^D=D(x)-D(G(x)),\ for\ D(x),D(G(x))\in [0,1]
$$

この式は、識別器が、正しいサンプルを間違いだと誤解する可能性(最初の項)や、  
偽のサンプルを本物だと間違う可能性(2番目の項)を最小化しようとしていることを示している。  
  
生成器の損失関数は、次のようになる。  
$$
J_G=-J^D
$$
  
#### 非飽和GAN(NS-GAN)
このNS-GANでは、2つの損失関数が直接対抗するものとして定式化されずに、2つの独立した損失関数として定義される。  
$$
J^G=E_{z\sim p_g}\log{[D(G(x))]}\\
J^D=E_{x\sim p_r}\log{[D(x)]}+E_{z\sim p_g}\log{[1-D(G(x))]}
$$

NS-GANはMM-GANなどと比べ初期の訓練が速く進むという特徴を持つ。  
ただし、なぜうまく収束するかは理論的には説明できない。  
  
#### Wassertein GAN
- 損失関数を大幅に改善し、理解可能で明確な停止条件を与えてくれる
- WGANは実験的にもよりよい結果を出力する傾向がある
- GANに関するこれまでの研究と違い、損失関数から話題が始まって、なぜKLダイバージェンスが理論的・実験的な観点から  
うまく正当化できていないかを示している。  
  
WGANは生成サンプルの見た目の品質と明確な相関のあるearth mover's distanceを損失関数としている。  
識別器のWassertein損失関数は、
$$
\max E_{x\sim P_r}[f_w(x)]-E_{z\sim p(z)}[f_w(g_{\theta}(z))]
$$

となる。  
ここでは、識別器として働く関数$(f_w)$がある。  
鑑定士はearth mover's distanceを推定し、これは関数$f_w$のこれまでとは異なったパラメータ化のもとで、  
本物(最初の項)と、生成されたもの(2番目の項)の分布間の最大損失を計算している。  
鑑定士は、$f_w$を使って2つのパラメータ空間を共有空間に投影し、生成器が動かさなければならない確率質量が  
できるだけ大きくなるように最適化を行う。  
生成器を表す式は、
$$
\min E_{x\sim P_r}[f_w(x)]-E_{z\sim p_z}[f_w(g_{\theta}(z))]
$$
  
であり、本物の分布である期待値と、生成された分布である期待値の間の距離を最小化しようとしている。  
  
生成器が解こうとする問題は、以下のものである。  
1. 真の分布$x$を($x\sim P_r$)、または生成された分布($z\sim p(z)$下で定義される$g_{\theta}(z)$)から$x^*$を得る
2. 潜在空間からサンプルされた$z$は$g_{\theta}$によって変換され、$x$と同じ空間サンプル$x^*$となる  
これは$f_w$を使って評価される   
3. 損失関数(この場合はearth mover's distanceを使った損失関数)を最小化する  
  
WGANには2つの実践的意義がある。  
- このGANには明確な停止基準を持っている。  
それは、識別器の損失関数と見た目の品質の間に相関があることが最近の論文で示されている。  
Wasserstein距離を測れば、いつ停止するかについて教えてくれる。  
- WGANが収束するように訓練を行うことができる。  

#### 訓練上の工夫
#### 入力の正規化
画素値を-1から1の間の値に正規化すると、その他の機械学習と同じように計算を容易に行うことができる。  
  
#### バッチ正規化
ミニバッチを、各層へ入力するたびに正規化する。  
  
#### 勾配制約
勾配が大きすぎる場合は、何か良くないことが起こっているという考え方に基づいている。  
  
#### 識別器を多く訓練する
2つのやり方がある。  
- 生成器が何も生成しないうちに、識別器をよく学習させておく
- 訓練サイクル内で識別器をより多く更新する。  
一般的には、生成器の更新1回あたり、識別器は5回更新する